In [1]:
// --- Day 23: A Long Walk ---

// Puzzle description redacted as-per Advent of Code guidelines

// You may find the puzzle description at: https://adventofcode.com/2023/day/23

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 23);
WriteLines(inputLines);

Loading puzzle file: Day23.txt
Total lines: 141
Max line length: 141

#.###########################################################################################################################################
#...###...#...#.......#...#.....#.........#...#...#...#...#...#.....#...#.......#...#...#.......#.....#...........#.....#...###...#.....#...#
###.###.#.#.#.#.#####.#.#.#.###.#.#######.#.#.#.#.#.#.#.#.#.#.#.###.#.#.#.#####.#.#.#.#.#.#####.#.###.#.#########.#.###.#.#.###.#.#.###.#.#.#
#...#...#...#.#.#.....#.#.#...#.#.......#.#.#.#.#.#.#.#.#.#.#.#.#...#.#.#...#...#.#.#.#.#.#.....#.#...#.........#.#...#.#.#...#.#.#...#.#.#.#
#.###.#######.#.#v#####.#.###.#.#######.#.#.#.#.#.#.#.#.#.#.#.#.#.###.#.###.#.###.#.#.#.#.#.#####.#.###########.#.###.#.#.###.#.#.###.#.#.#.#


In [4]:
string[] testInputLines = [
    "#.#####################",
    "#.......#########...###",
    "#######.#########.#.###",
    "###.....#.>.>.###.#.###",
    "###v#####.#v#.###.#.###",
    "###.>...#.#.#.....#...#",
    "###v###.#.#.#########.#",
    "###...#.#.#.......#...#",
    "#####.#.#.#######.#.###",
    "#.....#.#.#.......#...#",
    "#.#####.#.#.#########v#",
    "#.#...#...#...###...>.#",
    "#.#.#v#######v###.###v#",
    "#...#.>.#...>.>.#.###.#",
    "#####v#.#.###v#.#.###.#",
    "#.....#...#...#.#.#...#",
    "#.#########.###.#.#.###",
    "#...###...#...#...#.###",
    "###.###.#.###v#####v###",
    "#...#...#.#.>.>.#.>.###",
    "#.###.###.#.###.#.#v###",
    "#.....###...###...#...#",
    "#####################.#",
];

In [5]:
record Point(int x, int y) 
{
    public Point Up() => new(x, y - 1);
    public Point Down() => new(x, y + 1);
    public Point Left() => new(x - 1, y);
    public Point Right() => new(x + 1, y);   
}

record Tile(Point point, char type);

In [6]:
// A full journey; every step from start to end
class HikeJourney(HashSet<Point> steps, Point start, Point end)
{
    public HashSet<Point> Steps => steps;

    public Point Start => start;

    public Point End => end;

    public int Distance => Steps.Count - 1; 

    public HikeJourney AddStep(Point step)
    {
        HashSet<Point> newSteps = new(Steps);
        newSteps.Add(step);
        return new(newSteps, Start, step);
    }
}

// A summarisation of a journey, having decomposed the
// tile map into a graph of paths / junctions (nodes). 
// Required to efficiently search the map in part 2
class NodeWalk
{
    public Point Current { get; }
    public int Distance { get; init; }
    public HashSet<Point> VisitedNodes { get; }
    List<HikeJourney> Journeys { get; }

    public NodeWalk(Point start)
    {
        Current = start;
        Distance = 0;
        VisitedNodes = [start];
        Journeys = new();
    }

    public NodeWalk AddJourney(HikeJourney journey) => new(this, journey);

    NodeWalk(NodeWalk current, HikeJourney journey)
    {
        Current = journey.End;
        Distance = current.Distance + journey.Distance;
        VisitedNodes = new(current.VisitedNodes.Append(journey.End));
        Journeys = new(current.Journeys.Append(journey));
    }

    public HikeJourney ToJourney()
    {
        HashSet<Point> allSteps = new(Journeys.SelectMany(j => j.Steps));
        var j = new HikeJourney(allSteps, Journeys.First().Start, Journeys.Last().End);
        return j;
    }
}

class HikeSim 
{
    protected Dictionary<Point, Tile> tiles;
    private Point startPoint;
    private Point endPoint;
    protected int xMax;
    protected int yMax;

    public HikeSim(string[] inputLines)
    {
        var rows = inputLines.Length;
        var cols = inputLines[0].Length;

        tiles = new(rows * cols);

        for (var row = 0; row < rows; row++)
        {
            var rowStr = inputLines[row];
            for (var col = 0; col < cols; col++) {
                Point tilePoint = new(col, row);
                Tile tile = new(tilePoint, rowStr[col]);

                tiles[tilePoint] = tile;
            }
        }

        xMax = cols;
        yMax = rows;

        startPoint = new(1, 0); // Same in sample and actual

        var endRow = yMax - 1;
        var endCol = inputLines[endRow].IndexOf('.'); // Assume only one point available on end row
        endPoint = new(endCol, endRow);
    }

    public NodeWalk FindLongest()
    {
        var allPaths = DiscoverAllPaths();
        
        Queue<NodeWalk> queue = new();

        var initial = new NodeWalk(startPoint);
        var longest = initial;
        queue.Enqueue(initial);

        int processed = 0;
        while (queue.TryDequeue(out var nodeWalk))
        {
            processed++;
            if (processed % 1_000_000 == 0)
            {
                Console.WriteLine($"Processed {processed} steps. Queue length: {queue.Count}");
            }

            if (nodeWalk.Current == endPoint)
            {
                if (nodeWalk.Distance > longest.Distance)
                {
                    longest = nodeWalk;
                    Console.WriteLine($"New longest steps found at {longest.Distance} steps.");
                }
            }
            else
            {
                var nextJourneys = allPaths[nodeWalk.Current];
                foreach (var nextJourney in nextJourneys)
                {
                    if (!nodeWalk.VisitedNodes.Contains(nextJourney.End))
                    {
                        var newWalk = nodeWalk.AddJourney(nextJourney);
                        queue.Enqueue(newWalk);
                    }
                }
            }
        }

        return longest;
    }

    // Discover all paths from a given point to next junctions / terminations
    Dictionary<Point, List<HikeJourney>> DiscoverAllPaths()
    {
        Dictionary<Point, List<HikeJourney>> allPaths = new();
        
        Queue<Point> queue = new();
        queue.Enqueue(startPoint);
        allPaths[startPoint] = new();

        while (queue.TryDequeue(out var node))
        {
            foreach (var next in DiscoverPaths(node))
            {
                allPaths[node].Add(next);
                
                if (!allPaths.Keys.Contains(next.End))
                {
                    queue.Enqueue(next.End);
                    allPaths[next.End] = new();
                }
            }
        }

        return allPaths;
    }

    IEnumerable<HikeJourney> DiscoverPaths(Point node)
    {
        foreach (var nodeStep in GetValidOptions(node))
        {
            HikeJourney currentJourney = new([node, nodeStep], node, nodeStep);

            while (true)
            {
                // Keep walking along the path until we hit the end or a junction
                List<Point> options = new();
                foreach (var opt in GetValidOptions(currentJourney.End))
                {
                    var alreadyStepped = currentJourney.Steps.Contains(opt);
                    if (alreadyStepped) continue;
                    
                    options.Add(opt);
                }

                if (options.Count == 1)
                {
                    // Continuation of the current path
                    currentJourney = currentJourney.AddStep(options[0]);
                }
                else
                {
                    // We have reached the end of this path
                    yield return currentJourney;
                    break;
                }
            }
        }
    }

    protected IEnumerable<Point> GetValidOptions(Point point)
    {
        foreach (var opt in GetOptions(point))
        {
            var inBounds = opt switch 
            {
                {x: < 0} => false,
                {y: < 0} => false,
                var p when p.x >= xMax => false,
                var p when p.y >= yMax => false,
                _ => true
            };

            if (!inBounds) continue;

            var canStep = tiles[opt].type != '#';
            if (!canStep) continue;

            yield return opt;
        }
    }

    protected virtual IEnumerable<Point> GetOptions(Point p)
    {
        // There's a map of nearby hiking trails (your puzzle input) that
        // indicates paths (.), forest (#), and steep slopes (^, >, v, and <).
        
        var tile = tiles[p];

        return tile.type switch 
        {
            '.' => [p.Up(), p.Down(), p.Left(), p.Right()],
            '^' => [p.Up()],
            '>' => [p.Right()],
            '<' => [p.Left()],
            'v' => [p.Down()],
            _ => []
        };
    }
}

In [7]:
// This hike contains 94 steps. (The other possible hikes you could have taken were 90, 86, 82, 82, and 74 steps long.)

var testInputHike = new HikeSim(testInputLines);
var testInputResult = testInputHike.FindLongest();
Console.WriteLine(testInputResult.Distance);

New longest steps found at 90 steps.
New longest steps found at 94 steps.
94


In [8]:
// Find the longest hike you can take through the hiking trails listed on your map. How many steps long is the longest hike?

var part1Hike = new HikeSim(inputLines);
var part1Longesst = part1Hike.FindLongest();
var part1Answer = part1Longest.Distance;
Console.WriteLine(part1Answer);

Error: (5,19): error CS0103: The name 'part1Longest' does not exist in the current context

In [9]:
// 2210 is correct!
Ensure(2210, part1Answer);

Error: (2,14): error CS0103: The name 'part1Answer' does not exist in the current context

In [10]:
// --- Part Two ---

// Puzzle description redacted as-per Advent of Code guidelines

// You may find the puzzle description at: https://adventofcode.com/2023/day/23

In [11]:
class HikeSim2 : HikeSim
{
    public HikeSim2(string[] inputLines): base(inputLines) {}

    protected override IEnumerable<Point> GetOptions(Point p)
    {
        // There's a map of nearby hiking trails (your puzzle input) that
        // indicates paths (.), forest (#), and steep slopes (^, >, v, and <).
        
        var tile = tiles[p];

        return tile.type switch 
        {
            '.' or '^' or '>' or '<' or 'v' => [p.Up(), p.Down(), p.Left(), p.Right()],
            _ => []
        };
    }

    public IEnumerable<string> Render(HikeJourney journey)
    {
        foreach (var y in Enumerable.Range(0, yMax))
        {
            char[] row = new char[xMax];
            foreach (var x in Enumerable.Range(0, xMax))
            {
                Point point = new(x, y);
                row[x] = tiles[point].type;
                if (journey.Steps.Contains(point)) {
                    row[x] = 'O';
                }
            }
            yield return new String(row);
        }
    }
}

In [12]:
// In the example above, this increases the longest hike to 154 steps:

var testInputHike2 = new HikeSim2(testInputLines);
var testInputResult2 = testInputHike2.FindLongest();
Console.WriteLine(testInputResult2.Distance);


New longest steps found at 90 steps.
New longest steps found at 94 steps.
New longest steps found at 118 steps.
New longest steps found at 126 steps.
New longest steps found at 154 steps.
154


In [13]:
// Find the longest hike you can take through the surprisingly dry hiking trails listed on your map. How many steps long is the longest hike?

var part2Hike = new HikeSim2(inputLines);
var part2Longest = part2Hike.FindLongest();
var part2Answer = part2Longest.Distance;
Console.WriteLine(part2Answer);

New longest steps found at 2210 steps.
New longest steps found at 2614 steps.
New longest steps found at 2730 steps.
New longest steps found at 2778 steps.
New longest steps found at 2918 steps.
New longest steps found at 2998 steps.
New longest steps found at 3038 steps.
New longest steps found at 3086 steps.
New longest steps found at 3158 steps.
New longest steps found at 3174 steps.
New longest steps found at 3226 steps.
New longest steps found at 3238 steps.
New longest steps found at 3254 steps.
New longest steps found at 3306 steps.
New longest steps found at 3378 steps.
New longest steps found at 3474 steps.
New longest steps found at 3494 steps.
New longest steps found at 3546 steps.
New longest steps found at 3562 steps.
New longest steps found at 3582 steps.
New longest steps found at 3662 steps.
New longest steps found at 3742 steps.
New longest steps found at 3790 steps.
Processed 1000000 steps. Queue length: 675680
New longest steps found at 3858 steps.
New longest steps 

In [14]:
// 6522 is correct!
Ensure(6522, part2Answer);

In [15]:
foreach (var line in part2Hike.Render(part2Longest.ToJourney()))
{
    Console.WriteLine(line);
}

#O###########################################################################################################################################
#OOO###OOO#OOO#OOOOOOO#...#.....#OOOOOOOOO#OOO#OOO#OOO#OOO#OOO#OOOOO#...#.......#OOO#OOO#OOOOOOO#OOOOO#OOOOOOOOOOO#OOOOO#OOO###OOO#OOOOO#OOO#
###O###O#O#O#O#O#####O#.#.#.###.#O#######O#O#O#O#O#O#O#O#O#O#O#O###O#.#.#.#####.#O#O#O#O#O#####O#O###O#O#########O#O###O#O#O###O#O#O###O#O#O#
#OOO#OOO#OOO#O#O#OOOOO#.#.#...#.#OOOOOOO#O#O#O#O#O#O#O#O#O#O#O#O#OOO#.#.#...#...#O#O#O#O#O#OOOOO#O#OOO#OOOOOOOOO#O#OOO#O#O#OOO#O#O#OOO#O#O#O#
#O###O#######O#O#O#####.#.###.#.#######O#O#O#O#O#O#O#O#O#O#O#O#O#O###.#.###.#.###O#O#O#O#O#O#####O#O###########O#O###O#O#O###O#O#O###O#O#O#O#
#OOO#OOO#OOOOO#O#O>.###.#.#...#.#OOOOO#O#O#O#O#O#O#O#O#O#OOO#OOO#OOO>.#...#.#...#O#OOO#OOO#O#OOO#O#OOOOO#OOOOOOO#O###O#O#O#OOO#O#O###O#O#O#O#
###O###O#O#####O#O#.###.#.#.###.#O###O#O#O#O#O#O#O#O#O#O###########O#####.#.###.#O#########O#O#O#O#####O#O#######O###O#O#O#O###O#O###O#O#O#O#
#OOO#O